In [2]:
#!pip install numpy
#!pip install torch
import numpy #1.24
print(numpy.__version__)

#!pip install transformers[torch]
#!pip install transformers
#!pip install scikit-learn
#!pip install sentencepiece
#!pip install protobuf

#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

#!pip install tf-keras

1.26.4


In [3]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU disponivel')
    print('Número de GPUs disponíveis:', torch.cuda.device_count())
    print('Nome da GPU:', torch.cuda.get_device_name(0))

else:
    device = torch.device('cpu')
    print('GPU nao disponivel, utilizando CPU')

GPU disponivel
Número de GPUs disponíveis: 1
Nome da GPU: NVIDIA GeForce RTX 2070


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('output_deberta.csv', sep=';')

inputs = [str(text) for text in data['texto'].tolist()]

outputs = data['sentiment'].tolist()

train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader


class Dataset(Dataset):
    def __init__(self, inputs, outputs, tokenizer, max_length=512):
        self.inputs = inputs
        self.outputs = outputs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        label = self.outputs[idx]

        input_encoding = self.tokenizer(
            input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label)  # Certificar que os labels sejam tensores
        }
    

model_name = 'mrm8488/deberta-v3-ft-financial-news-sentiment-analysis'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis", num_labels=3).to(device)


train_dataset = Dataset(train_inputs, train_outputs, tokenizer)
test_dataset = Dataset(test_inputs, test_outputs, tokenizer)

c:\Users\igorm\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\igorm\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\igorm\.cache\huggingface\hub\models--mrm8488--deberta-v3-ft-financial-news-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [13]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import os

training_args = TrainingArguments(
    output_dir='/training_args/results_dir',
    num_train_epochs=4,
    evaluation_strategy="steps",
    per_device_train_batch_size=16,  # Increased batch size
    per_device_eval_batch_size=16,  # Increased batch size
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/training_args/logs',
    logging_steps=10,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=2  # Accumulate gradients
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)



trainer.train()

model_save_path = "./results_model/_model"
tokenizer_save_path = "./results_model/tokenizer"


os.makedirs(model_save_path, exist_ok=True)
os.makedirs(tokenizer_save_path, exist_ok=True)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Modelo salvo em {model_save_path}")
print(f"Tokenizador salvo em {tokenizer_save_path}")

  6%|▌         | 10/172 [00:36<07:26,  2.76s/it]

{'loss': 0.5169, 'grad_norm': 30.085582733154297, 'learning_rate': 9e-07, 'epoch': 0.23}


                                                
  6%|▌         | 10/172 [00:41<07:26,  2.76s/it]

{'eval_loss': 0.127266064286232, 'eval_runtime': 5.2974, 'eval_samples_per_second': 65.693, 'eval_steps_per_second': 4.153, 'epoch': 0.23}


 12%|█▏        | 20/172 [01:08<06:57,  2.75s/it]

...

{'loss': 0.2681, 'grad_norm': 14.916300773620605, 'learning_rate': 1.6800000000000002e-05, 'epoch': 3.91}


                                                 
 99%|█████████▉| 170/172 [09:32<00:05,  2.94s/it]

{'eval_loss': 0.2658576965332031, 'eval_runtime': 5.285, 'eval_samples_per_second': 65.846, 'eval_steps_per_second': 4.163, 'epoch': 3.91}


100%|██████████| 172/172 [09:39<00:00,  3.37s/it]


{'train_runtime': 579.1723, 'train_samples_per_second': 9.586, 'train_steps_per_second': 0.297, 'train_loss': 0.3576463450526082, 'epoch': 3.95}
Modelo salvo em ./results_model/_model
Tokenizador salvo em ./results_model/tokenizer
